In [52]:
import sys
import os
import re
import csv
import pandas as pd
import subprocess as sp
from pprint import pprint
from pathlib import Path
import yaml

In [53]:
run_dir = Path("/home/roman/dev/gpgpusims/run")
# run_dir = Path("/Users/roman/dev/gpgpusims/run")

In [ ]:
def parse_hw_csv_2(csv_file, hw_data, appargs, kdata):
    cfg = None

    return
    with open(csv_file, 'rU') as data_file:
        print("Parsing HW csv file {0}".format(csv_file))
        reader = csv.reader(data_file)
        state = "start"
        header = []
        kcount = 0
        for row in reader:
            if state == "start":
                if len(row) == 0:
                    continue
                if "ID" == row[0]:
                    state = "kernel_proc"
                continue
            if state == "kernel_proc":
                if len(row) == 1:
                    print("Bad line - possibly the app failed -- {0}".format(row))
                    break
                metric = row[-3]
                if metric == "device__attribute_display_name":
                    cfg = row[-1]
                    continue

                try:
                    value = float(row[-1].replace(",",""))
                except ValueError:
                    print("Bad line - possibly the app failed -- {0}".format(row))
                    continue

                if len(kdata) <= kcount:
                    kdata.append({})
                if metric not in kdata[kcount]:
                    kdata[kcount][metric] = []
                kdata[kcount][metric].append(value)
                kcount = int(row[0]) + 1

        print("Kernels found: {0}".format(kcount))

    if cfg != "" and cfg != None:
        cfg = re.sub(r" \(\d+\)$","", cfg) # Strip off the (0), (1), etc that some profiler versions put on the end of the device name
        if cfg not in hw_data:
            hw_data[cfg] = {}
        hw_data[cfg][appargs] = kdata

In [ ]:
def parse_hw_csv(csv_file, appargs, kdata):
    cfg = ""
    cfg_col = None
    hw_data = {}

    with open(csv_file, 'rU') as data_file:
        print("Parsing HW csv file {0}".format(csv_file))
        
        # print(data_file.read())
        # return
        
        # line must match r"^==\d*==\s*Profiling result:\s*$"
        # line must match r"^==PROF== Disconnected\s*$"
        
        reader = csv.reader(data_file)
        state = "start"
        header = []
        kcount = 0
        for row in reader:
            print(row)
            # Begin by searching for the text line that indicates the beginning of
            # the profile dump
            if state == "start" and len(row) > 0:
                if "Profiling result" in row[0] or "==PROF== Disconnected" in row[0]:
                    state = "header_proc"
                continue

            # The frist line is a header line what indicates the place of each
            # stat on the next line
            if state == "header_proc":
                if "Event result" in row[0]:
                    continue
                header = row
                count = 0

                # get the device name column - which is a special attribute
                for elem in row:
                    if elem == "Device":
                        cfg_col = count
                    elif elem == "device__attribute_display_name":
                        cfg_col = count
                    count += 1

                state = "kernel_proc"
                continue

            # The next sequence of lines are all the kernel launches with the values for each stat
            if state == "kernel_proc":
                if len(row) == 1:
                    print("Bad line - possibly the app failed -- {0}".format(row))
                    break

                # skip the memcopies
                if "[CUDA " in "".join(row):
                    continue

                if cfg_col == None:
                    continue

                # Skip lines without a device listed
                if row[cfg_col] == "":
                    continue

                # Set the Device
                if cfg != "" and cfg != row[cfg_col]:
                    print("data for more than one device in {0}..{1}:{2}"\
                        .format(csv_file,cfg,elem))
                    exit()

                cfg = row[cfg_col]
                count = 0
                if len(kdata) <= kcount:
                    kdata.append({})
                for elem in row:
                    if header[count] not in kdata[kcount]:
                        kdata[kcount][header[count]] = []
                    print("header[count]", header[count])
                    print("kdata[kcount][header[count]", kdata[kcount][header[count]])
                    try:
                        value = float(elem.replace(",",""))
                        kdata[kcount][header[count]].append(value)
                    except ValueError:
                        if "n/a" != elem:
                            kdata[kcount][header[count]].append(elem)
                    count += 1
                    
#                logger.log("Kernel Launch {0}: HW Kernel found".format(kcount))
                kcount += 1
                continue
        
        print("Kernels found: {0}".format(kcount))
        
    if cfg != "" and cfg != None:
        # Strip off the (0), (1), etc that some profiler versions 
        # put on the end of the device name
        cfg = re.sub(r" \(\d+\)$","", cfg)
        if cfg not in hw_data:
            hw_data[cfg] = {}
        hw_data[cfg][appargs] = kdata
        
    return hw_data

In [54]:
def convert_hw_csv(csv_file, output_csv_file):
    with open(csv_file, 'rU') as f:
        reader = csv.reader(f)
        
        # find the start of the csv dump
        prof_pat = re.compile(r"^==\d*==\s*Profiling result:\s*$")
        prof_abort = re.compile(r"^==PROF== Disconnected\s*$")
        for row in reader:
            line = ", ".join(row)
            # print(line)
            if prof_pat.match(line) is not None:
                # print("found start")
                break
            if prof_abort.match(line) is not None:
                # print("found abort")
                break
        
        os.makedirs(output_csv_file.parent, exist_ok=True)
        with open(output_csv_file, 'w') as out:
            output_writer = csv.writer(out)

            # write the valid csv rows to new file
            for row in reader:
                output_writer.writerow(row)

In [55]:
# parse hw stats
hw_results = run_dir / "native" / "matrixmul" / "nvidia_geforce_gtx_1080" / "results"
assert hw_results.is_dir()
hw_stat_file = hw_results / "result.txt"
hw_stat_cycles_file = hw_results / "result.cycles.txt"

hw_stat_file_out = hw_results / "result.csv"
hw_stat_cycles_file_out = hw_results / "result.cycles.csv"

In [56]:
convert_hw_csv(hw_stat_file, hw_stat_file_out)
convert_hw_csv(hw_stat_cycles_file, hw_stat_cycles_file_out)

In [59]:
df = pd.read_csv(hw_stat_file_out)
df

,Start,Duration,Grid X,Grid Y,Grid Z,Block X,Block Y,Block Z,Registers Per Thread,Static SMem,Dynamic SMem,Size,Throughput,SrcMemType,DstMemType,Device,Context,Stream,Name,Correlation_ID
0,us,us,NaN,NaN,NaN,NaN,NaN,NaN,NaN,KB,B,KB,GB/s,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,330085.887000,1.024000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.000000,3.725290,Pinned,Device,NVIDIA GeForce GTX 1080 (0),1.0,14.0,[CUDA memcpy HtoD],123.0
2,330093.919000,1.920000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.000000,1.986821,Pinned,Device,NVIDIA GeForce GTX 1080 (0),1.0,14.0,[CUDA memcpy HtoD],124.0
3,330124.767000,4.864000,1.0,1.0,1.0,32.0,32.0,1.0,30.0,8.000000,0,NaN,NaN,NaN,NaN,NVIDIA GeForce GTX 1080 (0),1.0,14.0,_Z13MatrixMulCUDAILi32EEvPfS0_S0_ii,127.0
4,330152.288000,1.088000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.000000,3.506156,Device,Pinned,NVIDIA GeForce GTX 1080 (0),1.0,14.0,[CUDA memcpy DtoH],132.0


In [58]:
df = pd.read_csv(hw_stat_cycles_file_out)
df

,Device,Context,Stream,Kernel,Correlation_ID,elapsed_cycles_sm,inst_per_warp,branch_efficiency,warp_execution_efficiency,warp_nonpred_execution_efficiency,...,dram_read_transactions,dram_write_transactions,dram_read_throughput,dram_write_throughput,dram_utilization,half_precision_fu_utilization,ecc_transactions,ecc_throughput,dram_read_bytes,dram_write_bytes
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,%,%,%,...,NaN,NaN,MB/s,B/s,NaN,NaN,NaN,B/s,NaN,NaN
1,NVIDIA GeForce GTX 1080 (0),1.0,14.0,_Z13MatrixMulCUDAILi32EEvPfS0_S0_ii,127.0,152810.0,138.0,100.000000,100.000000,98.550725,...,76.0,0.0,740.528715,0.000000,Low (1),Idle (0),0.0,0.000000,2432.0,0.0
